In [79]:
import pandas as pd

In [80]:
house_df = pd.read_csv("preprocessed_data.csv")
house_df = house_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
print(house_df.shape)
print(house_df.columns)
print(house_df.head())
print(house_df.info())


(4417, 18)
Index(['list_date', 'address.lat', 'address.lon', 'lot_size.size', 'prop_type',
       'year_built', 'beds', 'price', 'baths_full', 'baths_half',
       'is_new_construction', 'address.neighborhood_name',
       'building_size.size', 'garage', 'Zip Code', 'Population',
       '% Unemployment Rate', 'Avg. Income/H/hold'],
      dtype='object')
   list_date  address.lat  address.lon  lot_size.size prop_type  year_built  \
0     737854    39.926237   -75.176307          700.0     condo      1920.0   
1     737838    40.069313   -75.075879         3171.0     condo      1957.0   
2     737826    40.001261   -75.117655          602.0     condo      1920.0   
3     737816    39.987430   -75.100497          909.0     condo      1920.0   
4     737749    39.937969   -75.179954         1044.0     condo      2020.0   

   beds     price  baths_full  baths_half  is_new_construction  \
0   3.0  160000.0         1.0         0.0                    0   
1   3.0  265000.0         1.0        

In [81]:
# Import school data
school_df = pd.read_excel('SPR_SY1819_School_Metric_Scores_20200127.xlsx', sheet_name='SPR SY2018-2019')

sdf = school_df[['Zip Code', 'Overall Pts Earn']]
print(sdf.head())
print('')
sdf = sdf[sdf['Overall Pts Earn']!='Insufficient Data']
sdf = sdf[sdf['Zip Code']!='Closed']
sdf['Overall Pts Earn'] = sdf['Overall Pts Earn'].astype(str).astype(float)

# Get average of ratings by zip code
sdf_avg = sdf.groupby('Zip Code').mean().reset_index()
sdf_avg.rename(columns={'Overall Pts Earn': 'Avg_Pts_Earn'}, inplace=True)
sdf_avg['Zip Code']=sdf_avg['Zip Code'].astype(int)
print(sdf_avg)
print(sdf_avg.shape)
print(sdf_avg['Avg_Pts_Earn'].min(), sdf_avg['Avg_Pts_Earn'].max())


  Zip Code Overall Pts Earn
0    19142            18.93
1    19139            21.26
2    19104            21.75
3    19104            40.20
4    19139            11.36

    Zip Code  Avg_Pts_Earn
0      19102     46.360000
1      19103     55.006000
2      19104     42.652000
3      19106     44.566000
4      19111     44.233333
5      19114     54.833333
6      19115     81.190000
7      19116     60.924286
8      19118     53.720000
9      19119     43.640000
10     19120     49.194167
11     19121     28.481538
12     19122     29.131429
13     19123     45.984444
14     19124     33.797059
15     19125     38.668889
16     19126     37.633333
17     19127     52.310000
18     19128     43.452857
19     19129     38.395000
20     19130     39.707500
21     19131     35.030000
22     19132     29.490000
23     19133     29.842500
24     19134     34.798182
25     19135     39.366667
26     19136     39.896000
27     19137     57.316000
28     19138     30.911250
29     19139     33.0

In [82]:
# import hospital data
hos_df = pd.read_excel('Hospital General Information.xlsx')
hos_df = hos_df[hos_df['County Name'] == 'PHILADELPHIA']
hdf = hos_df[['ZIP Code','Hospital overall rating']]
print(hdf)
print(hdf.shape)

hdf_sum = hdf.groupby('ZIP Code').count().reset_index()
hdf_sum.rename(columns={'Hospital overall rating': 'Num_Hospitals'}, inplace=True)
print(hdf_sum)
print(hdf_sum.shape)

      ZIP Code Hospital overall rating
3505     19122           Not Available
3506     19118                       2
3507     19140                       2
3541     19111                       3
3556     19104                       4
3560     19114                       2
3573     19141                       2
3588     19107                       3
3603     19152                       3
3609     19104                       4
3611     19107                       3
3627     19102                       2
3629     19128                       3
3631     19124           Not Available
3660     19104           Not Available
3662     19134           Not Available
3664     19104           Not Available
(17, 2)
    ZIP Code  Num_Hospitals
0      19102              1
1      19104              4
2      19107              2
3      19111              1
4      19114              1
5      19118              1
6      19122              1
7      19124              1
8      19128              1
9      191

In [83]:
# merge data

# left join the house sale data with school data
merged_school = pd.merge(left=house_df, right=sdf_avg, how='left', left_on='Zip Code', right_on='Zip Code')
print(merged_school.head())
# Left join house sale data with hospital data
merged_hos = pd.merge(left=merged_school, right=hdf_sum, how='left', left_on='Zip Code', right_on='ZIP Code')
print(merged_hos.head())


   list_date  address.lat  address.lon  lot_size.size prop_type  year_built  \
0     737854    39.926237   -75.176307          700.0     condo      1920.0   
1     737838    40.069313   -75.075879         3171.0     condo      1957.0   
2     737826    40.001261   -75.117655          602.0     condo      1920.0   
3     737816    39.987430   -75.100497          909.0     condo      1920.0   
4     737749    39.937969   -75.179954         1044.0     condo      2020.0   

   beds     price  baths_full  baths_half  is_new_construction  \
0   3.0  160000.0         1.0         0.0                    0   
1   3.0  265000.0         1.0         1.0                    0   
2   2.0   25000.0         1.0         0.0                    0   
3   3.0  265000.0         1.0         1.0                    0   
4   3.0  742500.0         3.0         1.0                    1   

              address.neighborhood_name  building_size.size  garage  Zip Code  \
0  South Philadelphia - Philadelphia,PA        

In [84]:
merged = merged_hos.drop('ZIP Code', axis = 1)
print(merged.columns)
# Fill NA with 0
merged['Num_Hospitals'] = merged['Num_Hospitals'].fillna(0)

merged.dropna(subset=['Avg_Pts_Earn'], inplace=True)
print(merged.isnull().sum())
print(merged.shape)
print(merged.info())

Index(['list_date', 'address.lat', 'address.lon', 'lot_size.size', 'prop_type',
       'year_built', 'beds', 'price', 'baths_full', 'baths_half',
       'is_new_construction', 'address.neighborhood_name',
       'building_size.size', 'garage', 'Zip Code', 'Population',
       '% Unemployment Rate', 'Avg. Income/H/hold', 'Avg_Pts_Earn',
       'Num_Hospitals'],
      dtype='object')
list_date                    0
address.lat                  0
address.lon                  0
lot_size.size                0
prop_type                    0
year_built                   0
beds                         0
price                        0
baths_full                   0
baths_half                   0
is_new_construction          0
address.neighborhood_name    0
building_size.size           0
garage                       0
Zip Code                     0
Population                   0
% Unemployment Rate          0
Avg. Income/H/hold           0
Avg_Pts_Earn                 0
Num_Hospitals             

In [85]:
merged.to_csv('pre_school_and_hos.csv', index=False)